# Production wire-scan data analysis
> 07.15.2021

In [ ]:
import sys
import importlib

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import proplot as plot
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

sys.path.append('/Users/46h/Research/code/')
from accphys.tools import utils
from accphys.tools import beam_analysis as ba 
from accphys.tools import plotting as myplt
from accphys.tools.accphys_utils import V_matrix_4x4_uncoupled
from accphys.tools.accphys_utils import Sigma_from_twiss2D
from accphys.tools.accphys_utils import normalize
from accphys.tools.accphys_utils import possible_points
from accphys.tools.accphys_utils import normalized_Sigma
from accphys.tools.emittance_measurement import reconstruct
from accphys.tools.emittance_measurement import read_pta
from accphys.tools.emittance_measurement import plot_profiles
from accphys.tools.emittance_measurement import get_sig_xy

In [ ]:
plot.rc['grid.alpha'] = 0.04
plot.rc['figure.facecolor'] = 'white'

## Load data 

In [ ]:
rec_node_id = 'Begin_Of_RTBT1'
# rec_node_id = 'Begin_Of_RTBT2'
# rec_node_id = 'RTBT_Diag:BPM19'
# rec_node_id = 'RTBT_Diag:WS20'
# rec_node_id = 'RTBT_Diag:BLM23'

In [ ]:
ws_ids = ['WS20', 'WS21', 'WS23', 'WS24']

In [ ]:
def is_harp_file(filename):
    file = open(filename, 'r')
    for line in file:
        if 'Harp' in line:
            return True
    return False

In [ ]:
exp_twiss_filename = 'data/twiss/model_twiss_{}.dat'.format(rec_node_id)
transfer_matrices_filename = 'data/transfer_matrix/model_transfer_mat_elems_default_{}.dat'.format(rec_node_id)
filenames = utils.list_files('./data/ws/')
filenames_ws = [filename for filename in filenames if not is_harp_file(filename)]
filenames_harp = [filename for filename in filenames if is_harp_file(filename)]

In [ ]:
filenames_ws

In [ ]:
filenames_harp

I don't currenlty understand the harp data format.

In [ ]:
# filename = filenames_harp[0]
# file = open(filename, 'r')
# for line in file:
#     print(line.rstrip())
# file.close()

In [ ]:
measurements = [read_pta(filename) for filename in filenames_ws]

In [ ]:
profile = measurements[0]['WS24']
display(profile.data)
display(profile.params)

In [ ]:
kws_raw = dict(legend=False, marker='.', ms=3, lw=0)
kws_fit = dict(legend=False, color='k', alpha=0.2, zorder=0)

In [ ]:
axes = plot_profiles(measurements, ws_ids, show_fit=False, kws_raw=kws_raw, kws_fit=kws_fit)
plt.savefig('figures/profiles.png', dpi=300, facecolor='white')

In [ ]:
# axes = profile_data.plot_profiles(measurements, ws_ids, show_fit=True, kws_raw=kws_raw, kws_fit=kws_fit)

## Reconstruct transverse covariance matrix

Collect the measured beam moments at each wire-scanner.

In [ ]:
diag_wire_angle = np.radians(-45.0)

In [ ]:
moments = {ws_id: [] for ws_id in ws_ids}
for ws_id in ws_ids:
    for measurement in measurements:
        profile = measurement[ws_id]
        sig_xx = profile.get_param('Sigma', dim='x')**2
        sig_yy = profile.get_param('Sigma', dim='y')**2
        sig_uu = profile.get_param('Sigma', dim='u')**2
        sig_xy = get_sig_xy(sig_xx, sig_yy, sig_uu, diag_wire_angle)
        moments[ws_id].append([sig_xx, sig_yy, sig_xy])
    moments[ws_id] = np.array(moments[ws_id])

In [ ]:
fig, axes = plot.subplots(ncols=4, figsize=(9, 2))
plt_kws = dict(marker='.')
for ws_id, ax in zip(ws_ids, axes):
    ax.plot(moments[ws_id][:, 0], **plt_kws)
    ax.plot(moments[ws_id][:, 1], **plt_kws)
    ax.plot(moments[ws_id][:, 2], **plt_kws)
axes.format(ylabel=r'[mm$^2$]', xlabel='Measurement index', toplabels=ws_ids)
ax.legend(labels=[r'$\langle{x^2}\rangle$', r'$\langle{y^2}\rangle$', r'$\langle{xy}\rangle$'],
          ncols=1, loc=(1.02, 0), fontsize='small')
plt.savefig('figures/moments.png', dpi=300, facecolor='white')

Collect the linear transfer matrices at each wire-scanner.

In [ ]:
transfer_mats = {ws_id: [] for ws_id in ws_ids}
for ws_index, ws_id in enumerate(ws_ids):
    if ws_id == 'WS02': # don't have data from this wire-scanner
        continue
    for _ in range(len(measurements)):
        matrix_elements = np.loadtxt(transfer_matrices_filename)[ws_index]
        transfer_mats[ws_id].append(matrix_elements.reshape((4, 4)))

Reconstruct the covariance matrix at the RTBT entrance.

In [ ]:
moments_list, transfer_mats_list = [], []
for ws_id in ws_ids:
    moments_list.extend(moments[ws_id])
    transfer_mats_list.extend(transfer_mats[ws_id])

In [ ]:
Sigma = reconstruct(transfer_mats_list, moments_list)
print('Sigma:')
print(Sigma)
alpha_x, alpha_y, beta_x, beta_y, eps_x, eps_y = ba.get_twiss2D(Sigma)
twiss = (alpha_x, alpha_y, beta_x, beta_y)
eps_1, eps_2 = ba.intrinsic_emittances(Sigma)
print('eps_4D = {:.3f}'.format(np.sqrt(np.linalg.det(Sigma))))
print('eps_1, eps_2 = {:.3f}, {:.3f}'.format(eps_1, eps_2))
print('eps_x, eps_y = {:.3f}, {:.3f}'.format(eps_x, eps_y))
print('alpha_x, alpha_y = {:.3f}, {:.3f}'.format(alpha_x, alpha_y))
print('beta_x, beta_y = {:.3f}, {:.3f}'.format(beta_x, beta_y))

## Discussion 

In [ ]:
correlation_matrix = utils.cov2corr(Sigma)
print(correlation_matrix)

In [ ]:
coupling_coeff = 1.0 - np.sqrt((eps_1 * eps_2) / (eps_x * eps_y))
print('Coupling coefficient = {}'.format(coupling_coeff))

Below are the rms ellipses defined by the covariance matrix. The cross-plane projections are squares in reality.

In [ ]:
myplt.rms_ellipses(Sigma, fill=True)
plt.savefig('figures/rms_ellipses.png', dpi=300, facecolor='white');

In [ ]:
# Expected covariance matrix at RTBT entrance (from default optics)
twiss_exp = np.loadtxt(exp_twiss_filename)
alpha_x_exp, alpha_y_exp, beta_x_exp, beta_y_exp = twiss_exp
Sigma_exp = Sigma_from_twiss2D(alpha_x_exp, alpha_y_exp, beta_x_exp, beta_y_exp, eps_x, eps_y)

In [ ]:
def plot_lines(ax, transfer_mats, moments, dim='x', twiss=None, **plt_kws):  
    for transfer_mat, (sig_xx, sig_yy, sig_xy) in zip(transfer_mats, moments):
        h_pts, v_pts = possible_points(transfer_mat, sig_xx, sig_yy, dim, twiss)
        ax.plot(h_pts, v_pts, **plt_kws)

In [ ]:
def plot_reconstructed_phasespace(Sigma, Sigma_exp=None, twiss=None, scale=2.0):
    fig, axes = plot.subplots(ncols=2, sharex=False, sharey=False, figsize=(6, 2.5))

    # Plot reconstructed ellipses
    if twiss is not None:
        Sigma = normalized_Sigma(Sigma, *twiss)
    angle_xxp, cx, cxp = myplt.rms_ellipse_dims(Sigma, 'x', 'xp')
    angle_yyp, cy, cyp = myplt.rms_ellipse_dims(Sigma, 'y', 'yp')
    myplt.ellipse(axes[0], 2 * cx, 2 * cxp, angle_xxp, lw=2)
    myplt.ellipse(axes[1], 2 * cy, 2 * cyp, angle_yyp, lw=2)

    # Plot design ellipses
    plot_exp = Sigma_exp is not None
    if plot_exp:
        if twiss is not None:
            Sigma_exp = normalized_Sigma(Sigma_exp, *twiss)
        angle_xxp_exp, cx_exp, cxp_exp = myplt.rms_ellipse_dims(Sigma_exp, 'x', 'xp')
        angle_yyp_exp, cy_exp, cyp_exp = myplt.rms_ellipse_dims(Sigma_exp, 'y', 'yp')
        myplt.ellipse(axes[0], 2 * cx_exp, 2 * cxp_exp, angle_xxp_exp, lw=1, alpha=0.25)
        myplt.ellipse(axes[1], 2 * cy_exp, 2 * cyp_exp, angle_yyp_exp, lw=1, alpha=0.25)
        
    # Plot possible points at reconstruction location as lines. 
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    kws = dict(lw=0.3)
    for ws_id, color in zip(ws_ids, colors):
        plot_lines(axes[0], transfer_mats[ws_id], moments[ws_id], 'x', twiss, color=color, **kws)
        plot_lines(axes[1], transfer_mats[ws_id], moments[ws_id], 'y', twiss, color=color, **kws)

    # Legends
    custom_lines_1 = [Line2D([0], [0], color='black', lw=2)]
    labels_1 = ['calc']
    if plot_exp:
        custom_lines_1.append(Line2D([0], [0], color='black', lw=1, alpha=0.25))
        labels_1.append('exp')
    custom_lines_2 = [Line2D([0], [0], color=color) for color in colors[:4]]
    axes[1].legend(custom_lines_1, labels_1, ncols=1, loc=(1.02, 0.8), fontsize='small')
    axes[1].legend(custom_lines_2, ws_ids, ncols=1, loc=(1.02, 0), fontsize='small')

    # Formatting
    axes.format(grid=False, xlabel_kw=dict(fontsize='large'), ylabel_kw=dict(fontsize='large'))
    if twiss:
        axes.format(aspect=1)
        labels = [r"$x_n$ [mm]", r"$x'_n$ [mrad]", r"$y_n$ [mm]", r"$y'_n$ [mrad]"]
    else:
        labels = ["x [mm]", "x' [mrad]", "y [mm]", "y' [mrad]"]
        
    max_coords = 2.0 * np.sqrt(np.diag(Sigma))
    xmax, xpmax, ymax, ypmax = scale * max_coords
    axes[0].format(xlim=(-xmax, xmax), ylim=(-xpmax, xpmax), xlabel=labels[0], ylabel=labels[1])
    axes[1].format(xlim=(-ymax, ymax), ylim=(-ypmax, ypmax), xlabel=labels[2], ylabel=labels[3])
    return axes

In [ ]:
axes = plot_reconstructed_phasespace(Sigma, Sigma_exp)
axes.format(suptitle='Reconstructed phase space')
plt.savefig('figures/lines.png', dpi=300, facecolor='white')

In [ ]:
axes = plot_reconstructed_phasespace(Sigma, twiss=twiss)
axes.format(suptitle='Reconstructed phase space (normalized by measured Twiss)')
plt.savefig('figures/lines_norm.png', dpi=300, facecolor='white')